# Moving Average

**参考资源**

> - [指数加权移动平均(Exponential Weighted Moving Average)](https://blog.csdn.net/mzpmzk/article/details/80085929)
> - [How Is the Exponential Moving Average (EMA) Formula Calculated?](https://www.investopedia.com/ask/answers/122314/what-exponential-moving-average-ema-formula-and-how-ema-calculated.asp)
> - [How to Calculate Exponential Moving Averages](https://sciencing.com/calculate-exponential-moving-averages-8221813.html)
> - [Simple, Exponential, and Weighted Moving Averages](https://www.thebalance.com/simple-exponential-and-weighted-moving-averages-1031196)
> - [Moving Average, Weighted Moving Average, and Exponential Moving Average](https://www.investopedia.com/ask/answers/071414/whats-difference-between-moving-average-and-weighted-moving-average.asp)
> - [How Is the Exponential Moving Average (EMA) Formula Calculated?](https://www.investopedia.com/ask/answers/122314/what-exponential-moving-average-ema-formula-and-how-ema-calculated.asp)
> - [Exploring the Exponentially Weighted Moving Average](https://www.investopedia.com/articles/07/ewma.asp)
> - [为什么在优化算法中使用指数加权平均](https://www.jianshu.com/p/41218cb5e099)

# SMA

# WMA

$
WMA = \frac{P_1n+P_2(n-1)+\cdots+P_n}{\frac{n(n+1)}{2}}
$

# EMA

$
EMA=P_t K + EMA_{t-1}(1-K)
$

> - K: $\frac{2}{N+1}$ N表示周期内的数量（天数）

# EWMA

$
V_t=\beta V_{t-1} + (1 - \beta)\theta_t
$

> - $\beta$： 大于等于0.9 （0.9表示周期为10天$\frac{1}{1-\beta}=\frac{1}{1-0.9}=10$；0.98表示周期50天）